In [1]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objs as go
import pandas as pd
import plotly
import plotly.plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

rank = pd.read_csv('../data/rank_file.csv')

# Create a list of indicators 
available = list(rank.columns.values)
for i in ['Unnamed: 0','City','State','Population','Natural_total_rank', 'Human_related_rank', 'Economy_rank', 'Tertiary_Rank', 'Latitude', 'Longitude']:
    available.remove(i)
    
# Create a list of labels for dropdown
labels = ['Fewer Crimes','More Hospitals','Early Education Options',
          'University Options','Air Quality','Water Quality',
           'Fewer Toxics','Fewer Hazardous Particles','Green Coverage',
           'Employment Rate','Sales Revenue','Income','Tuition Affordability',
          'Bars','Restaurants','Museums','Libraries','Parks','Top Restaurants']

# Put available and labels in a two-dimensional list
pairs = [available,labels]

In [ ]:
app = dash.Dash()
alldata = rank
# Get DataFrame value

app.layout = html.Div([
    
    html.Div([
        html.Br(),
        html.Br(),
        html.Center(html.H1("Find Your Dream City to Live in the US", 
                            style={'color': 'lavender', 'fontFamily': 'Helvetica', 'fontSize': 50}
                           ),
                   ),
        
        html.Center(
            html.P("Powerd by City Fynders",
                           style={'color': 'black','fontFamily':'Helvetica', 'fontSize': 20}),
                   ),
        html.Br(),
        html.Br()],
        style={'backgroundColor': 'FireBrick', 'margin': 0,'padding': 0}),
        
        html.Div([
        html.Br(),
        html.Br(),
        html.Center(html.Img(src="http://travelquaz.com/wp-content/uploads/2015/09/photo-travel-holidays-to-the-usa-america-visit-usa-images.jpg"))
        ]),
       

        
        # First important factor
        html.Div([             
            
            html.P('Important No.1'),
            
            dcc.Dropdown(
                id='First-care',
                # Set the labels and values 
                options=[ {'label': pairs[1][i],'value': pairs[0][i]} for i in range(19)],
            )
            ],
        style={'width': '20%', 'fontFamily':'Helvetica','display': 'inline-block'}),
        
        
        # Second important factor
        html.Div([
            
            html.P('Important No.2'),
            
            dcc.Dropdown(
                id='Second-care',
                options=[{'label': pairs[1][i],'value': pairs[0][i]} for i in range(19)],

            )
            ],
        style={'width': '20%','fontFamily':'Helvetica', 'display': 'inline-block'}),
        
        # Third important factor
        html.Div([
            
            html.P('Important No.3'),
            
            dcc.Dropdown(
                id='Third-care',
                options=[{'label': pairs[1][i],'value': pairs[0][i]} for i in range(19)],

            )
            ],
        style={'width': '20%','fontFamily':'Helvetica', 'display': 'inline-block'}),
        
        # Fourth important factor
        html.Div([
            
            html.P('Important No.4'),
            
            dcc.Dropdown(
                id='Fourth-care',
                options=[{'label': pairs[1][i],'value': pairs[0][i]} for i in range(19)],

            )
            ],
        style={'width': '20%','fontFamily':'Helvetica', 'display': 'inline-block'}),
        
        # Fifth important factor
        html.Div([
            
            html.P('Important No.5'),
            
            dcc.Dropdown(
                id='Fifth-care',
                options=[{'label': pairs[1][i],'value': pairs[0][i]} for i in range(19)],

            )
            ],
        style={'width': '20%', 'fontFamily':'Helvetica','display': 'inline-block'}),
        
        html.Br(),
        html.Br(),
        html.Br(),
        
        html.Center(html.Button('Go Searching!', id='button',
                                style={'width': 200, 'color': 'white', 'backgroundColor': 'FireBrick', 'height': 70,'fontSize': 20})),
        
        html.Div(dcc.Graph(id='indicator-graphic'))
    ])
    #dcc.Graph(id='indicator-graphic')


@app.callback(
    dash.dependencies.Output('indicator-graphic', 'figure'),
    [dash.dependencies.Input('button', 'n_clicks')],
    [dash.dependencies.State('First-care', 'value'),
     dash.dependencies.State('Second-care', 'value'),
     dash.dependencies.State('Third-care', 'value'),
     dash.dependencies.State('Fourth-care', 'value'),
     dash.dependencies.State('Fifth-care', 'value'),
    ]
)
    
def update_graph(Button, First_care, Second_care, Third_care, Fourth_care, Fifth_care):
    df = pd.DataFrame()
    df['City'] = rank['City']
    df['First'] = rank[First_care]
    df['Second'] = rank[Second_care]
    df['Third'] = rank[Third_care]
    df['Fourth'] = rank[Fourth_care]
    df['Fifth'] = rank[Fifth_care]
    df['Total'] = (df['First']*5+df['Second']*4+df['Third']*3+df['Fourth']*2+df['Fifth']*1).rank(ascending=1)
    df = df.sort_values('Total', ascending=1)
    df['reverse_rank'] = df['Total'].rank(ascending=0)
    df['text'] = df['City'] + '<br># Final Rank ' + (df['Total']).astype(str)
    df['longitude'] = alldata['Longitude']
    df['latitude'] = alldata['Latitude']
    
    limits = [(0,10),(10,20),(20,30),(30,40),(40,50)]
    colors = ["rgb(0,116,217)","rgb(255,65,54)","rgb(133,20,75)","rgb(255,133,27)","lightgrey"]
    cities = []


    for i in range(len(limits)):
        lim = limits[i]
        df_sub = df[lim[0]:lim[1]]
        city = dict(
            type = 'scattergeo',
            locationmode = 'USA-states',
            lon = df_sub['longitude'],
            lat = df_sub['latitude'],
            text = df_sub['text'],
            marker = dict(
                size = df_sub['reverse_rank']*15,
                color = colors[i],
                line = dict(width=0.5, color='rgb(40,40,40)'),
                sizemode = 'area'
            ),
            name = '{0} - {1}'.format(lim[0],lim[1]) )
        cities.append(city)

        layout = dict(
            title = 'Your Dream City Results',
            showlegend = True,
            geo = dict(
                scope='usa',
                projection=dict( type='albers usa' ),
                showland = True,
                landcolor = 'rgb(217, 217, 217)',
                subunitwidth=1,
                countrywidth=1,
                subunitcolor="rgb(255, 255, 255)",
                countrycolor="rgb(255, 255, 255)"
            ),
        )
        fig = dict( data=cities, layout=layout )
        
    return {   
        plotly.offline.plot( fig, validate=False )     
    }

app.run_server()

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [29/Nov/2017 18:25:19] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [29/Nov/2017 18:25:19] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [29/Nov/2017 18:25:19] "GET / HTTP/1.1" 200 -
[2017-11-29 18:25:19,911] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/anaconda/lib/python3.6/site-packages/flask/app.py", line 1982, in wsgi_app
    response = self.full_dispatch_request()
  File "/anaconda/lib/python3.6/site-packages/flask/app.py", line 1614, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/anaconda/lib/python3.6/site-packages/flask/app.py", line 1517, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/anaconda/lib/python3.6/site-packages/flask/_compat.py", line 33, in reraise
    raise value
  File "/anaconda/lib/python3.6/site-packages/flask/app.py", line 1612, in full_dispatch_request
    rv = self.d